# Chapter 19. Functional programming techniques

**Session 28**

**Agenda**

- **Housekeeping**: notes & code, expensing food, start recording
- **Recap**
    - 18 thinking functionally
        - writing code w/o side effects
        - using functions
- **Today:** 
    - chapter 19 key concepts - techniques in functional programming 
- **For Next time:** 
    - chapter 20 - Blending OOP and functional
    
This chapter covers:

* higher-order functions, currying, and partial application
* Persistent data structures
* Lazy evaluation and lazy lists as generalizing Java
streams
* Pattern matching and how to simulate it in Java
* Referential transparency and caching

### 19.1 Functions

Functions are first class citizens in Java.

In [ ]:
import java.util.function.*;
import java.util.stream.*;

In [ ]:
UnaryOperator<Integer> squareIt = x -> x * x;
System.out.println(
    squareIt.apply(5)
);
Function<String, Integer> strToInt = Integer::parseInt;
System.out.println(
    strToInt.apply("5")
);

### 19.1.1  Higher order functions

Functions that take another function as a parameter or return a function as a result

In [ ]:
var myList = List.of(1, 2, 4, 8);
myList.stream().map(squareIt).collect(Collectors.toList());

In [ ]:
Function<Integer, Stream<Integer>> prevAndNext = i -> List.of(i - 1, i, i + 1).stream();
myList.stream()
    .map(squareIt)
    .flatMap(prevAndNext)
    .collect(Collectors.toList());

### 19.1.2 Currying

Let's practice by converting Celsius to Farenheit.

Unit conversion always involves a conversion factor and, from time to time, a base-
line adjustment factor. The formula to convert Celsius to Fahrenheit, for example, is
CtoF(x) = x*9/5 + 32 . 
The basic pattern of all unit conversion is as follows:
  - Multiply by the conversion factor.
  - Adjust the baseline if relevant.

In [ ]:
double converter(double x, double f, double b) {
    return x * f + b;
}

var celsius = 100;
converter(celsius, 1.8, 32);

In [ ]:
UnaryOperator<Double> curriedConverter(Double f, Double b) {
    return x -> x * f + b;
}
var cToF = curriedConverter(1.8, 32d);
var usdToGbp = curriedConverter(0.6, 0d);
var kmToMi = curriedConverter(0.6214, 0d);

var value = cToF.apply(100d);
System.out.println(value);

Given general types:

A f(B, C, D) = (B -> A) f(C, D)

In fact, any function can be represented as a chain of curried calls.  [See Haskell](https://wiki.haskell.org/Currying#:~:text=In%20Haskell%2C%20all%20functions%20are,(b%20%2D%3E%20c)%20.)

### 19.2 Persistent data structures

In [ ]:
class TrainJourney {
    public int price;
    public TrainJourney onward;
    public TrainJourney(int p, TrainJourney t) {
        price = p;
        onward = t;
    }
}

> suppose that a variable firstJourney contains the route
from X to Y and that a variable secondJourney contains the route from Y to Z. If you
call link(firstJourney, secondJourney) , this code destructively updates first-
Journey to also contain secondJourney , so in addition to the single user who
requests a trip from X to Z seeing the combined journey as intended, the journey
from X to Y has been updated destructively. Indeed, the firstJourney variable is no
longer a route from X to Y, but one from X to Z, which breaks code that depends on
firstJourney ’s not being modified!

In [ ]:
TrainJourney link(TrainJourney a, TrainJourney b){
    if (a==null) return b;
        TrainJourney t = a;
    while(t.onward != null){
        t = t.onward;
    }
    t.onward = b;
    return a;
}

In [ ]:
TrainJourney append(TrainJourney a, TrainJourney b) {
    return a==null ? b : new TrainJourney(a.price, append(a.onward, b));
}

Pure functional data structures use recursion and don't allow for mutation of state.

### 19.3 Lazy evaluation with Streams

Let's get a list of prime numbers.

In [ ]:
public boolean isPrime(int candidate) {
    int candidateRoot = (int) Math.sqrt((double) candidate);
    return IntStream.rangeClosed(2, candidateRoot)
        .noneMatch(i -> candidate % i == 0);
}

public Stream<Integer> primes(int n) {
    return Stream.iterate(2, i -> i + 1)
        .filter(x -> isPrime(x))
        .limit(n);
}

In [ ]:
primes(7).collect(Collectors.toList())

> You have to iterate through every number
every time to see whether it can be divided by a candidate number. (In fact, you need
only test numbers that have been already classified as prime.)
Ideally, the stream should filter out numbers that are divisible by the prime that
the stream is producing on the go

In [ ]:
// get stream of numbers
IntStream numbers(){
    return IntStream.iterate(2, n -> n + 1);
}

// take the head
int head(IntStream numbers){
    return numbers.findFirst().getAsInt();
}

// get the tail
IntStream tail(IntStream numbers){
    return numbers.skip(1);
}

// recursively create stream of primes
IntStream primes2(IntStream numbers) {
    int head = head(numbers);
    return IntStream.concat(
        IntStream.of(head),
        primes(tail(numbers).filter(n -> n % head != 0))
    );
}

In [ ]:
primes2(numbers());

> you’re using two terminal operations to split the stream into its head and tail: find-
First and skip . Remember from chapter 4 that after you call a terminal operation on
a stream, it’s consumed forever!

> Lazy Evaluation: There’s an additional, more important problem : the method IntStream.concat expects two instances of a stream, but its second argument is a direct recursive call to primes , resulting in an infinite recursion! For many Java purposes, restrictions on Java 8 streams such as no recursive definitions are unproblematic and give your database-like queries expressivity and the ability to parallelize. Thus, the Java 8 designers chose a sweet spot. Nonetheless, the more general features and models of streams from functional languages such as Scala and Haskell can be useful additions to your programming toolbox. What you need is a way to lazily evaluate the call to the method.

> Functional programming techniques: primes in the second argument of concat . (In a more technical programming vocabulary, we refer to this concept as lazy evaluation, nonstrict evaluation or even call by name.) Only when you need to process the prime numbers (such as with the limit method) should the stream be evaluated. Scala (which we explore in chapter 20) provides support for this idea.

### 19.4 Pattern matching

Doesn't really exist in Java.  Switch expressions introduced in Java 13.

[Java switch: 4 wrongs don't make a right](https://blog.joda.org/2019/11/java-switch-4-wrongs-dont-make-right.html)

Java 14 updates usage of instanceOf to be more type safe:

https://blogs.oracle.com/javamagazine/pattern-matching-for-instanceof-in-java-14




In [ ]:
void outputToUppercase(Object obj) {
    if (obj instanceof String) {
        String str = (String) obj;
        System.out.println(str.toUpperCase());
    }
}

void outputToUppercase2(Object obj) {
    if (obj instanceof String str) {
        System.out.println(str.toUpperCase());
    }
}

In [ ]:
outputToUppercase2("abc");

### 19.5 Caching

> Even using (lock-protected) Hashtable or (concurrent-without-locking) ConcurrentHashMap instead of HashMap may not produce the expected performance if parallel calls are made.  Can be race conditions, which means that multiple processes might compute the same value to add to the map . Perhaps the best thing to take away from this struggle is the fact that mixing mutable state with concurrency is trickier than you’d imagine. Functional-style programming
avoids this practice except for low-level performance hacks such as caching. A second takeaway is that apart from implementing tricks such as caching, if you code in functional style, you never need to care whether another functional-style method that you call is synchronized, because you know that it has no shared mutable state.

### Combinators

i.e., Monads!

### Summary

- First-class functions are functions that can be passed as arguments, returned as results, and stored in data structures.
- A higher-order function takes one or more functions as input or returns another function. Typical higher-order functions in Java include comparing, andThen, and compose .
- Currying is a technique that lets you modularize functions and reuse code.
- A persistent data structure preserves the previous version of itself when it’s modified. As a result, it can prevent unnecessary defensive copying.  Streams in Java can’t be self-defined.
- A lazy list is a more-expressive version of a Java stream. A lazy list lets you produce elements of the list on demand by using a supplier that can create more of the data structure.
- Pattern matching is a functional feature that lets you unwrap data types. You can view data matching as generalizing Java’s switch statement.
- Referential transparency allows computations to be cached.
- Combinators are functional ideas that combine two or more functions